## 0. Libraries

In [20]:
# libraries installation
!pip install tweet-preprocessor
!pip install emoji --upgrade
!pip install contractions
!pip install textblob

Requirement already up-to-date: emoji in /opt/anaconda3/lib/python3.7/site-packages (0.5.4)


In [21]:
# basic libraries
import pandas as pd
import pickle
import sklearn
import numpy as np
import string
import re
from collections import Counter
import time
import matplotlib.pyplot as plt
import io
import nltk
from nltk.corpus import stopwords
import os
from textblob import TextBlob

## 1. Data loading (with tweets_sample dataset start from clean tweets section)

### 538 tweets

In [22]:
#read all csv in folder data
data_dir = '../data/'
start = True

# what percentage of each csv of tweets you want in your data (1% should give us aroun 100K tweets)
#fraction = .2


for root,dirs,files in os.walk(data_dir):
    for file in files:
        if file.endswith(".csv"):
            print(file)
            f = pd.read_csv(data_dir+file)
            #f = f.sample(frac = fraction)
            if start is True:
                data = f
                start = False
            else:
                data=data.append(f, sort=False)

IRAhandle_tweets_10.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IRAhandle_tweets_11.csv
IRAhandle_tweets_9.csv
IRAhandle_tweets_2.csv
IRAhandle_tweets_5.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IRAhandle_tweets_12.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IRAhandle_tweets_8.csv
IRAhandle_tweets_1.csv
IRAhandle_tweets_6.csv
IRAhandle_tweets_7.csv
IRAhandle_tweets_4.csv
IRAhandle_tweets_13.csv
IRAhandle_tweets_3.csv


In [23]:
#keep only english tweets

data = data[data.language=='English']
data = data.dropna(axis=0,subset=['content'])
print(f'Number of tweets: {data.shape[0]}')

Number of tweets: 2116866


In [24]:
tweets_orig = data

# keep only original tweets (if you uncomment lines below)

#tweets_orig = data[data.post_type!='RETWEET']
#tweets_orig = tweets_orig[tweets_orig.post_type!='QUOTE_TWEET']
#tweets_orig = tweets_orig.reset_index(drop=True)

In [25]:
print(f'number of original tweets: {tweets_orig.shape[0]}')


number of original tweets: 2116866


In [26]:
tweets_orig.columns

Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'harvested_date', 'following', 'followers', 'updates',
       'post_type', 'account_type', 'retweet', 'account_category',
       'new_june_2018', 'alt_external_id', 'tweet_id', 'article_url',
       'tco1_step1', 'tco2_step1', 'tco3_step1'],
      dtype='object')

In [27]:
tweets_orig = tweets_orig[tweets_orig.account_category!='NonEnglish']

In [28]:
#tweets_orig.to_csv('tweets_sample')

## 2. Clean tweets

In [29]:
# import the functions for preprocessing
import tweets_preprocessing 

In [30]:
# list of symbols to remove
my_symbols = list(string.punctuation)
my_symbols.remove("'")
my_symbols.append('…')
my_symbols.append('»')
my_symbols.append('«')
my_symbols.append('„')
my_symbols.append('“')

# load stop words
nltk.download('stopwords')

# load all stop words in the three languages
all_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
tweets_orig = tweets_orig.dropna(axis=0,subset=['content'])

In [32]:
# clean the tweets, returns tweets_words (list of lists, each word in each tweet)
# hashtags (list of hashtags in each tweet) 
tweets_words, hashtags, mentions, emojis = tweets_preprocessing.preprocessor(tweets_orig.content, 
                                                                             my_symbols,
                                                                             all_stopwords)

In [1]:
positive_hash = 'hillaryforpr, imwithher2016, imwithhillary, hillaryforpresident, hillaryforamerica, hereiamwithher, estoyconella, hillarysopresidential, uniteblue, hillstorm2016,bluewave2016, welovehillary, itrusther, bluewave, hrcisournominee, itrusthillary, standwithmadampotus, momsdemandhillary, madamepresident, madampresident, imwither, herstory, republicans4hillary, hillarysoqualified, werewithher, vote4hillary, strongertogether, readyforhillary, hillafornia, unitedagainsthate, votehillary, wearewithher, republicansforhillary, hrc2016, connecttheleft, yeswekaine, voteblue2016, hillary2016, sheswithus, hillyes, iamwithher, heswithher, voteblue, hillaryaprovenleader, imwiththem, bernwithher, ohhillyes, imwithher, clintonkaine2016, whyimwithher, turnncblue, hillarystrong'
positive_hash = positive_hash.split(', ')

In [2]:
positive_hash = set(['#'+has for has in positive_hash]+['#gohillary',
 '#hillaryclinton16',
 '#hillaryclintonforpresident',
 '#ilovehillary',
 '#republicansforhillary',
 '#readyforhillary',
 '#votehillary',
 '#girls4hillary'])

In [3]:
negative_hash = 'hillarysolympics, hillaryforprison, hillaryforprison2016, moretrustedthanhillary, heartlesshillary, neverclinton, handcuffhillary, queenofcorrupton, crookedhiliary, nomoreclintons, hillary4jail, fbimwithher, clintoncrimefamily, hillno, queenofcorruption, hillarylostme, ohhillno, billclintonisrapist, democratliesmatter, lyingcrookedhillary, hypocritehillary, crookedclintons, hillarylies, neverhilary, shelies, releasethetranscripts, stophillary2016, riskyhillary, hillaryliedpeopledied, corrupthillary, hillary4prison2016, nohillary2016, wehatehillary, whatmakeshillaryshortcircuit, crookedhillaryclinton, deletehillary, dropouthillary, lyinhillary, hillaryliesmatter, nevereverhillary, stophillary, neverhilllary, clintoncorruption, clintoncrime, notwithher, hillary2jail, imnotwithher, lockherup, corruptclintons, indicthillary, sickhillary, crookedhilary, crookedhillary, hillaryrottenclinton, theclintoncontamination, lyinghillary, clintoncollapse, clintoncrimefoundation, neverhillary, criminalhillary, crookedclinton, hillary4prison, killary, iwillneverstandwithher'
negative_hash = negative_hash.split(', ')

In [4]:
negative_hash = set(['#'+has for has in negative_hash] + ['#neverhillary',
 '#thingsmoretrustedthanhillary',
 '#crookedhillary',
 '#makeamoviehillary',
 '#sickhillary',
 '#thingshillarygoogles',
 '#zombiehillary',
 '#indicthillary',
 '#dropouthillary',
 '#releaseclintonsmedicalrecords',
 '#whyimnotvotingforhillary',
 '#billclintonisarapist',
 '#losinglikehillary',
 '#stophillary2016',
 '#herpeshillary',
 '#thingshillarywillneverhave',
 '#neverclinton',
 '#betternamesforhillarysbook',
 '#corrupthillary',
 '#nohillary',
 '#anybodybutclinton',
 '#noclintonsever',
 '#heilhillary',
 '#handcuffhillary',
 '#whatclintonwrites',
 '#nohillary2016',
 '#hypocritehillary',
 '#nevercrookedhillary',
 '#dontkillmehillary',
 '#billclintonrapist',
 '#3wordhillary',
 '#pedophilesforhillary',
 '#sexoffendersforhillary',
 '#satanists4hillary',
 '#arresthillary',
 '#lesshillarymorefun',
 '#libertynothillary',
 '#crookedclinton',
 '#assad_clinton',
 '#stopclinton',
 '#deplorablehillary',
 '#hillaryrottenclinton',
 '#saynotohillary',
 '#rejectedhillaryslogans',
 '#lyincrookedhillary',
 '#censoredforhillary',
 '#corruptclintons',
 '#unforgivablehillary',
 '#madhillary',
 '#whathillarywouldforadollar',
 '#impeachhillary',
 '#abusivehillary',
 '#stophillary',
 '#hackinghillary',
 '#arrestclinton',
 '#murderinghillary'])

In [5]:
all_hash = positive_hash.union(negative_hash)
hillary_clinton = set(['hillary','clinton'])

In [8]:
print(negative_hash)

{'#hillary4prison', '#hillaryliedpeopledied', '#hillaryrottenclinton', '#satanists4hillary', '#deplorablehillary', '#3wordhillary', '#censoredforhillary', '#releaseclintonsmedicalrecords', '#madhillary', '#hillaryforprison2016', '#hypocritehillary', '#imnotwithher', '#whyimnotvotingforhillary', '#lyinghillary', '#criminalhillary', '#crookedclinton', '#heartlesshillary', '#unforgivablehillary', '#thingshillarywillneverhave', '#hillarylostme', '#moretrustedthanhillary', '#crookedhilary', '#sickhillary', '#rejectedhillaryslogans', '#queenofcorruption', '#dropouthillary', '#shelies', '#crookedhiliary', '#notwithher', '#queenofcorrupton', '#lyincrookedhillary', '#neverclinton', '#nevercrookedhillary', '#wehatehillary', '#clintoncrime', '#lockherup', '#democratliesmatter', '#arrestclinton', '#arresthillary', '#impeachhillary', '#corruptclintons', '#indicthillary', '#lyingcrookedhillary', '#nomoreclintons', '#deletehillary', '#handcuffhillary', '#hackinghillary', '#libertynothillary', '#nohil

In [235]:
hashtags_list = []
for h in hashtags:
    hashtags_list.append(h.lower().split())
    
hillary_tweets = []
hillary_hashtags = []
hillary_tweet_meta = []
for tw,hashs,au,date,retweet,content in zip(tweets_words,hashtags_list,tweets_orig.author,tweets_orig.publish_date,tweets_orig.retweet,tweets_orig.content):
    j = len(set(tw) & hillary_clinton) + len(set(hashs) & all_hash)
    if j > 0:
        hillary_tweets.append(re.sub("[']", '',' '.join(tw)))
        hillary_hashtags.append(hashs)
        hillary_tweet_meta.append([au,re.sub("/", ' ',date),retweet,content])

## LDA

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [114]:
def lemm(doc):
    words = [t.lemma_ for t in nlp(doc) if t.lemma_ != '-PRON-']
    return ' '.join(words).lower()

In [117]:
import re
import spacy
import seaborn as sns
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
nlp = spacy.load('en_core_web_sm')

In [128]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=10, preprocessor = lemm)
tf = tf_vectorizer.fit_transform(hillary_tweets)
tf_feature_names = tf_vectorizer.get_feature_names()

In [104]:
no_topics = 15

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 15
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
sanders president bernie got obama need would million today meeting run dead assange flashback iowa
Topic 1:
american fake bill good trumps bush thats crimes thinker give north days needs yet weinstein
Topic 2:
says know make remember well day next gets every nothing law gave votes kkk wanted
Topic 3:
breaking comey one amp like election lynch win big let help money lies democratic people
Topic 4:
fbi russia white investigation look house congress email report probe doj latest emails much corrupt
Topic 5:
get still time back ever bad even old gun john wall guns ohio revealed gives
Topic 6:
debate wants email wikileaks cnn used via prison dnc party secret server team top private
Topic 7:
vote trump say im take stop really must way away attack says come war anyone
Topic 8:
bill trump deal chelsea via cant support corruption gop uranium speech rally foundation candidate show
Topic 9:
rt emails state clintons benghazi may made claims national says department dept scandal called se

In [68]:
no_topics = 20

# Run LDA
lda2 = LatentDirichletAllocation(n_components=no_topics, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 15
display_topics(lda2, tf_feature_names, no_top_words)

Topic 0:
hillary amp clinton want voters trump big see let wikileaks americans another country dems loss
Topic 1:
clinton people foundation say hillary it need show rally bill sessions dead left old pay
Topic 2:
hillary clinton end we turn msm biden telling feel killed least re line reports joe
Topic 3:
clinton back bill top stupid way scandal north nuclear gun lied sex office korea started
Topic 4:
hillary breaking trump supporters know got even doj lies race presidential million video meeting come
Topic 5:
clinton hillary trump crooked debate russia money deal chelsea foundation uranium speech obama cant bad
Topic 6:
hillary still corruption right may clinton much state national lying department hate goes special great
Topic 7:
hillary us going well next really trumps real every today twitter thats clinton needs muslim
Topic 8:
hillary clinton good times better collusion weinstein lynch tarmac trust reason loretta hope would working
Topic 9:
clinton says obama hillary win state suppo

In [129]:
no_topics = 5

# Run LDA
lda3 = LatentDirichletAllocation(n_components=no_topics, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 15
display_topics(lda3, tf_feature_names, no_top_words)

Topic 0:
hillary clinton bill obama one use call white report well house say good remember watch
Topic 1:
hillary clinton vote trump not obama want go election like win rt say would get
Topic 2:
clinton hillary trump sander video poll lie bill donald voter tell campaign time say try
Topic 3:
hillary clinton trump know give supporter campaign american pay fake right corruption work thing claim
Topic 4:
clinton hillary email rt new break fbi comey foundation state russia lynch investigation doj via


In [85]:
from sklearn.model_selection import GridSearchCV

In [125]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35], 'doc_topic_prior':[0.001,None]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, n_jobs = 5)

# Do the Grid Search
model.fit(tf)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [120]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(tf))

Best Model's Params:  {'doc_topic_prior': None, 'n_components': 5}
Best Log Likelihood Score:  -954065.2808361806
Model Perplexity:  932.3007538079268


## NMF

In [71]:
from sklearn.decomposition import NMF

In [ ]:
from gensim.corpora import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

# Use Gensim's NMF to get the best num of topics via coherence score
texts = [tw.split() for tw in hillary_tweets]

# Create a dictionary
# In gensim a dictionary is a mapping between words and their integer id
dictionary = Dictionary(texts)

# Filter out extremes to limit the number of features
dictionary.filter_extremes(
    no_below=3,
    no_above=0.85,
    keep_n=5000
)

# Create the bag-of-words format (list of (token_id, token_count))
corpus = [dictionary.doc2bow(text) for text in texts]

# Create a list of the topic numbers we want to try
topic_nums = list(np.arange(5, 75 + 1, 5))

# Run the nmf model and calculate the coherence score
# for each number of topics
coherence_scores = []

for num in topic_nums:
    nmf = Nmf(
        corpus=corpus,
        num_topics=num,
        id2word=dictionary,
        chunksize=2000,
        passes=5,
        kappa=.1,
        minimum_probability=0.01,
        w_max_iter=300,
        w_stop_condition=0.0001,
        h_max_iter=100,
        h_stop_condition=0.001,
        eval_every=10,
        normalize=True,
        random_state=42
    )
    
    # Run the coherence model to get the score
    cm = CoherenceModel(
        model=nmf,
        texts=texts,
        dictionary=dictionary,
        coherence='c_v'
    )
    
    coherence_scores.append(round(cm.get_coherence(), 5))

In [81]:
# Get the number of topics with the highest coherence score
scores = list(zip(topic_nums, coherence_scores))
best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]

print(best_num_topics)

NameError: name 'itemgetter' is not defined

In [127]:
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'alpha':[0.001,.01,.1]}

# Init the Model
nmf = NMF()

# Init Grid Search Class
model_nmf = GridSearchCV(nmf, param_grid=search_params, n_jobs = 5)

# Do the Grid Search
model_nmf.fit(tfidf)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=None, random_state=None, shuffle=False, solver='cd',
    tol=0.0001, verbose=0) does not.

In [339]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words = ['hillary','clinton'])
tfidf = tfidf_vectorizer.fit_transform(hillary_tweets)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
no_topics=12
nmf = NMF(n_components=no_topics, random_state=1, alpha=0, l1_ratio=.5, init='nndsvd').fit(tfidf)

display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
trump donald poll supporters debate russia vs says voters leads lead win rally wins like
Topic 1:
obama comey lynch fbi calls mueller congress house crimes america russia meeting congressman investigation abedin
Topic 2:
breaking doj deal plea offers expanding probe rumored take report video crooked grounds ample prosecuting
Topic 3:
sanders bernie debate iowa says democratic supporters campaign primary race voters clash california fight win
Topic 4:
bill remember campaign used hate flashback symbol made fire fury comments allwhite club joined golf
Topic 5:
vote women people didn black voting popular would want im never get please said america
Topic 6:
emails state foundation fbi email department via clintons dept release campaign deleted investigation wikileaks benghazi
Topic 7:
rt amp debalwaystrump https russia america realdonaldtrump rr mueller one think republicans nuclear benghazi would
Topic 8:
president qualified would never next one want run like become endorses becom

In [218]:
topic_share = np.matrix(nmf.transform(tfidf))

In [219]:
topics = np.argmax(topic_share, axis = 1)

In [342]:
len(set(results.author))

791

In [340]:
results = pd.DataFrame(hillary_tweet_meta, columns = ['author','date','is_retweet','content'])
results['hashtags'] = [' '.join(tw_hashtags) for tw_hashtags in hillary_hashtags]
results['topic'] = topics

In [314]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

In [315]:
from datetime import datetime
results.date = [datetime.strptime(d, '%m %d %Y %H:%M') for d in results.date]
results['year']=[d.year for d in results.date]
results['month']=[d.month for d in results.date]

In [316]:
res_grouped = results.groupby(['topic','month','year'], as_index=False)['author'].count()
res_total = results.groupby(['month','year'], as_index=False)['author'].count()

results = res_grouped.merge(res_total, on = ['month','year'])
results['total_tweets'] = results.author_x
results['tweets_share'] = results.author_x/results.author_y

del results['author_x']
del results['author_y']

In [317]:
all_combinations = pd.DataFrame([[topic, month, year] for topic in range(no_topics) 
                                 for month in range(1,13) 
                                 for year in [2015,2016,2017,2018]], 
                                columns = ['topic','month','year'])

In [318]:
results = results.merge(all_combinations, on=['topic','month','year'], how='outer')
results = results.fillna(0)

In [319]:
results['yearmonth'] = (results.year-2015)*12 + results.month

In [320]:
results.sort_values(by=['yearmonth','topic'], inplace=True)

In [337]:
results.to_csv('../results.csv')

In [457]:
results = pd.DataFrame(hillary_tweet_meta, columns = ['author','date','is_retweet','content'])
results['hashtags'] = [' '.join(tw_hashtags) for tw_hashtags in hillary_hashtags]
results['topic'] = topics

deff = np.array([1 for _ in range(len(results))])
deff[results.topic == 0] = 2
deff[results.topic == 2] = 2
deff[results.topic == 6] = 2
deff[results.topic == 5] = 0
deff[results.topic == 8] = 0

results['deff'] = deff
results = results[results.deff != 1]

res_grouped = results.groupby(['author','deff'], as_index=False)['topic'].count()
res_total = results.groupby(['author'], as_index=False)['topic'].count()

results = res_grouped.merge(res_total, on = ['author'])
results['tweets_share'] = results.topic_x/results.topic_y

results = results[results.topic_y > 25]

del results['topic_x']
del results['topic_y']

results.to_csv('../def_by_author.csv')

In [459]:
results = pd.DataFrame(hillary_tweet_meta, columns = ['author','date','is_retweet','content'])
results['hashtags'] = [' '.join(tw_hashtags) for tw_hashtags in hillary_hashtags]
results['topic'] = topics


res_grouped = results.groupby(['topic','is_retweet'], as_index=False)['author'].count()
res_total = results.groupby(['topic'], as_index=False)['author'].count()

results = res_grouped.merge(res_total, on = ['topic'])
results['tweets_share'] = results.author_x/results.author_y

results = results[results.author_y > 25]

del results['author_x']
del results['author_y']

results.to_csv('../ret_by_topic.csv')